In [2]:
import pandas as pd
import numpy as np


import tensorflow as tf
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.utils import to_categorical

from keras.models import model_from_json
from keras.layers import Flatten
from tensorflow.keras.callbacks import ModelCheckpoint

from keras.layers import Conv2D

In [3]:
train_dir = r'IE/train'
test_dir = r'IE/test'

SEED = 12
IMG_HEIGHT = 224
IMG_WIDTH = 224
BATCH_SIZE = 64
momentum = 0.9
EPOCHS = 500
#FINE_TUNING_EPOCHS = 30
LR = 0.001
NUM_CLASSES = 8
EARLY_STOPPING_CRITERIA=3
CLASS_LABELS  = ['Amusement', 'Anger', 'Awe', 'Contentment', 'Disgust', 'Excitement', 'Fear', 'Sadness']
CLASS_LABELS_EMOJIS = ["🥳", "😡", "😯", "😌", "🤢" ,"🤩", "😱" , "😔" ]

In [4]:
preprocess_fun = tf.keras.applications.resnet.preprocess_input

train_datagen = ImageDataGenerator(horizontal_flip=True,
                                   width_shift_range=0.1,
                                   height_shift_range=0.05,
                                   rotation_range= 10,
                                   rescale = 1./255,
                                   validation_split = 0,
                                   preprocessing_function=preprocess_fun
                                  )
test_datagen = ImageDataGenerator(rescale = 1./255,
                                  validation_split = 0,
                                  preprocessing_function=preprocess_fun)

train_generator = train_datagen.flow_from_directory(directory = train_dir,
                                                    target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = True , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    subset = "training",
                                                    seed = 12
                                                   )

test_generator = test_datagen.flow_from_directory(directory = test_dir,
                                                   target_size = (IMG_HEIGHT ,IMG_WIDTH),
                                                    batch_size = BATCH_SIZE,
                                                    shuffle  = False , 
                                                    color_mode = "rgb",
                                                    class_mode = "categorical",
                                                    seed = 12
                                                  )

Found 280 images belonging to 8 classes.
Found 280 images belonging to 8 classes.


In [5]:
def add_prefix(model, prefix: str, custom_objects=None):
    '''Adds a prefix to layers and model name while keeping the pre-trained weights
    Arguments:
        model: a tf.keras model
        prefix: a string that would be added to before each layer name
        custom_objects: if your model consists of custom layers you shoud add them pass them as a dictionary. 
            For more information read the following:
            https://keras.io/guides/serialization_and_saving/#custom-objects
    Returns:
        new_model: a tf.keras model having same weights as the input model.
    '''
    
    config = model.get_config()
    old_to_new = {}
    new_to_old = {}
    
    for layer in config['layers']:
        new_name = prefix + layer['name']
        old_to_new[layer['name']], new_to_old[new_name] = new_name, layer['name']
        layer['name'] = new_name
        layer['config']['name'] = new_name

        if len(layer['inbound_nodes']) > 0:
            for in_node in layer['inbound_nodes'][0]:
                in_node[0] = old_to_new[in_node[0]]
    
    for input_layer in config['input_layers']:
        input_layer[0] = old_to_new[input_layer[0]]
    
    for output_layer in config['output_layers']:
        output_layer[0] = old_to_new[output_layer[0]]
    
    config['name'] = prefix + config['name']
    new_model = tf.keras.Model().from_config(config, custom_objects)
    
    for layer in new_model.layers:
        layer.set_weights(model.get_layer(new_to_old[layer.name]).get_weights())
    
    return new_model

In [6]:
def pre_model_extractor(inputs):
    emotion_model = tf.keras.applications.resnet.ResNet101(input_shape=(224,224,3), include_top= False,
                                                                 weights= 'imagenet')

    def emotion_model_feature_extractor(inputs):
        z = emotion_model(inputs)
        y = tf.keras.layers.GlobalAveragePooling2D()(z)
        x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(y)
    
        return x, y, z

    # def mainobject_model_feature_extractor(inputs):
    #     y = mainobject_model(inputs)
    #     y = tf.keras.layers.GlobalAveragePooling2D()(y)
    #     x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax')(y)

    #     return x, y

    def classifier(features):
        x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', 
                                  name='classification')(features)
    
        return x
        
    def final_model(inputs, emotion_model, mainobject_model):
        x, emotion_features, z = emotion_model(inputs)
        mainobject_features = mainobject_model(inputs)
    
        merged_features = tf.concat([mainobject_features, emotion_features], axis=1)
        classification_output = classifier(merged_features)
    
        return classification_output, mainobject_features, emotion_features, z

    def define_compile_model(emotionweights_path):
    
        emotion_model = tf.keras.Model(inputs= inputs, outputs= emotion_model_feature_extractor(inputs))
        emotion_model.load_weights(emotionweights_path)
        mainobject_model = tf.keras.applications.resnet.ResNet101(input_shape=(224,224,3), include_top= True, 
                                                                         weights='imagenet')
        mainobject_model.trainable = False
    
        classification_output, mainobject_features, emotion_features, z = final_model(inputs, emotion_model, mainobject_model) 
        model = tf.keras.Model(inputs= inputs, outputs= classification_output)
        model.compile(optimizer=tf.keras.optimizers.SGD(0.001), 
                    loss='categorical_crossentropy',
                    metrics = ['acc'])
  
        return model, mainobject_features, emotion_features, z

    emotionweights_path = r'ResNet101-009-0.762864-0.650595.h5'
    model, mainobject_vector, GAP_features, emotion_tensor = define_compile_model(emotionweights_path)
    
    return model, mainobject_vector, GAP_features, emotion_tensor

    #return final_model(inputs, emotion_model, mainobject_model)

In [8]:
def classifier(features):
    x = tf.keras.layers.Dense(NUM_CLASSES, activation='softmax', 
                              name='classification')(features)
    return x

def crisscross_method(mainobject_tensor, emotion_tensor):
    mainobject_split = tf.split(mainobject_tensor, 128, axis=3)
    for i in range(len(mainobject_split)):
        reduced_tensor = tf.reduce_mean(mainobject_split[i], axis=3)
        #mainobject_split[i] = #tf.reshape(reduced_tensor, [reduced_tensor.shape[0], reduced_tensor.shape[1], 
                                                          #reduced_tensor.shape[2], 1])
        mainobject_split[i] = tf.expand_dims(reduced_tensor, axis=-1)
    reduced_mainobject_tensor = tf.concat(mainobject_split, axis=3)
    
    emotion_tensor_split = tf.split(emotion_tensor, 512, axis=3)
    for i in range(len(emotion_tensor_split)):
        reduced_tensor = tf.reduce_mean(emotion_tensor_split[i], axis=3)
#         emotion_tensor_split[i] = tf.reshape(reduced_tensor, (reduced_tensor.shape[0], reduced_tensor.shape[1], 
#                                                           reduced_tensor.shape[2], 1))
        emotion_tensor_split[i] = tf.expand_dims(reduced_tensor, axis=-1)
    reduced_emotion_tensor = tf.concat(emotion_tensor_split, axis=3)
    
    mainobject_split = tf.split(reduced_mainobject_tensor, 128, axis=3)
    emotion_tensor_split = tf.split(reduced_emotion_tensor, 128, axis=3)
    crisscross_split = []
    for i in range(len(mainobject_split)):
        crisscross_split.append(emotion_tensor_split[i])
        crisscross_split.append(mainobject_split[i])
    crisscross_tensor = tf.concat(crisscross_split, axis=3)
    
    return crisscross_tensor

def shallow_CNN(crisscross_tensor):
    x = Conv2D(1024, (3,3), strides =1, activation = 'relu')(crisscross_tensor)
    x = Conv2D(512, (3,3), strides=1, activation= 'relu')(x)
    x = Conv2D(512, (3,3), strides=1, activation= 'relu')(x)
    
    return x

def final_model(inputs, pre_model, mainobject_vector, GAP_features, emotion_tensor, mainobject2_model):
    #x = pre_model(inputs)
    mainobject_tensor = mainobject2_model(inputs)
    
    crisscross_tensor = crisscross_method(mainobject_tensor, emotion_tensor)
    shallow_features = shallow_CNN(crisscross_tensor)
    shallow_features = tf.reduce_mean(shallow_features, axis=1)
    shallow_features = tf.reduce_mean(shallow_features, axis=1)
    
    merged_features = tf.concat([mainobject_vector, shallow_features, GAP_features], axis=1)
    classification_output = classifier(merged_features)
    
    return classification_output

def define_compile_model(premodelweights_path):
    
    inputs = tf.keras.layers.Input(shape=(224,224,3))
    
    pre_model, mainobject_vector, GAP_features, emotion_tensor = pre_model_extractor(inputs)
    pre_model.load_weights(premodelweights_path)
    pre_model.trainable = False
    
    mainobject2_model = tf.keras.applications.resnet.ResNet101(input_shape=(224,224,3), include_top= False, 
                                                                 weights='imagenet')
    mainobject2_model = add_prefix(mainobject2_model, 'MainObject2_')
    mainobject2_model.trainable = False
    
    classification_output = final_model(inputs, pre_model, mainobject_vector, GAP_features, emotion_tensor, mainobject2_model) 
    model = tf.keras.Model(inputs= inputs, outputs= classification_output)
    model.compile(optimizer=tf.keras.optimizers.SGD(0.001), 
                loss='categorical_crossentropy',
                metrics = ['acc'])
  
    return model

In [10]:
premodelweights_path = r"Idea1_ResNet101-015-0.804192-0.662801.h5"
model = define_compile_model(premodelweights_path)
#model.load_weights(r'Idea1-001-0.742820-0.650900.h5')

model.summary()

Model: "model_7"
__________________________________________________________________________________________________
 Layer (type)                   Output Shape         Param #     Connected to                     
 input_5 (InputLayer)           [(None, 224, 224, 3  0           []                               
                                )]                                                                
                                                                                                  
 model_4 (Functional)           ((None, 8),          42674568    ['input_5[0][0]']                
                                 (None, 2048),                                                    
                                 (None, 7, 7, 2048)                                               
                                )                                                                 
                                                                                            

In [9]:
#for saving weights after every epoch
checkpoint = ModelCheckpoint('Idea3_ResNet101-{epoch:03d}-{acc:03f}-{val_acc:03f}.h5', verbose=1, 
                        monitor='val_acc',save_best_only=True, mode='auto') 
#earlystopping = tf.keras.callbacks.EarlyStopping(monitor='val_accuracy', mode='auto')

In [10]:
#training the model
history = model.fit(train_generator,
            batch_size = BATCH_SIZE,
            epochs= 100,
            validation_data = test_generator,
            callbacks= [checkpoint]
            ) 

history = pd.DataFrame(history.history)

Epoch 1/100
290/290 [==============================] - ETA: 0s - loss: 1.4169 - acc: 0.5047
Epoch 1: val_acc improved from -inf to 0.57827, saving model to Idea1.1_ResNet101-001-0.504715-0.578273.h5
290/290 [==============================] - 2094s 7s/step - loss: 1.4169 - acc: 0.5047 - val_loss: 1.2253 - val_acc: 0.5783
Epoch 2/100
290/290 [==============================] - ETA: 0s - loss: 1.0452 - acc: 0.6363
Epoch 2: val_acc improved from 0.57827 to 0.61153, saving model to Idea1.1_ResNet101-002-0.636295-0.611535.h5
290/290 [==============================] - 2086s 7s/step - loss: 1.0452 - acc: 0.6363 - val_loss: 1.1332 - val_acc: 0.6115
Epoch 3/100
290/290 [==============================] - ETA: 0s - loss: 0.9434 - acc: 0.6687
Epoch 3: val_acc improved from 0.61153 to 0.63015, saving model to Idea1.1_ResNet101-003-0.668732-0.630150.h5
290/290 [==============================] - 2086s 7s/step - loss: 0.9434 - acc: 0.6687 - val_loss: 1.0984 - val_acc: 0.6301
Epoch 4/100
290/290 [=======

KeyboardInterrupt: 

In [9]:
json_model = model.to_json()
with open('Idea3_ResNet101.json', 'w') as json_file:
    json_file.write(json_model)
print('Saved model.')

Saved model.
